In [1]:
from concurrent.futures.process import ProcessPoolExecutor

import satpy
import numpy as np
import matplotlib.pyplot as plt
import xarray
import pandas
import earthaccess
import os
from pathlib import Path
from datetime import datetime
import glob
from pykdtree.test_tree import test3d_8n_ub_leaf20
from skyfield.api import EarthSatellite
from skyfield.api import load, wgs84
from skyfield.iokit import parse_tle_file
from collections import defaultdict
from datetime import timedelta
from sat_orbits import *
from sat_info import *


In [7]:
# concatenate tle files together
tle_dir = Path("~/Downloads/").expanduser()

ts = load.timescale()
satellites = []

for tle_path in tle_dir.glob("sat00*"):
    print(tle_path)
    with load.open(str(tle_path)) as f:
        satellites.extend(parse_tle_file(f, ts))

/Users/cwelch/Downloads/sat000027424.txt
/Users/cwelch/Downloads/sat000049260.txt
/Users/cwelch/Downloads/sat000060989.txt
/Users/cwelch/Downloads/sat000040697.txt
/Users/cwelch/Downloads/sat000054234.txt
/Users/cwelch/Downloads/sat000043013.txt
/Users/cwelch/Downloads/sat000037849.txt
/Users/cwelch/Downloads/sat000039084.txt
/Users/cwelch/Downloads/sat000042063.txt
/Users/cwelch/Downloads/sat000025994.txt


In [8]:
satellites

[<EarthSatellite catalog #27424 epoch 2023-01-01 03:38:24 UTC>,
 <EarthSatellite catalog #27424 epoch 2023-01-01 14:03:18 UTC>,
 <EarthSatellite catalog #27424 epoch 2023-01-01 19:57:54 UTC>,
 <EarthSatellite catalog #27424 epoch 2023-01-01 21:41:14 UTC>,
 <EarthSatellite catalog #27424 epoch 2023-01-02 04:35:11 UTC>,
 <EarthSatellite catalog #27424 epoch 2023-01-02 14:47:22 UTC>,
 <EarthSatellite catalog #27424 epoch 2023-01-02 19:03:21 UTC>,
 <EarthSatellite catalog #27424 epoch 2023-01-02 20:59:23 UTC>,
 <EarthSatellite catalog #27424 epoch 2023-01-03 05:17:43 UTC>,
 <EarthSatellite catalog #27424 epoch 2023-01-03 10:24:20 UTC>,
 <EarthSatellite catalog #27424 epoch 2023-01-03 19:45:10 UTC>,
 <EarthSatellite catalog #27424 epoch 2023-01-04 00:59:33 UTC>,
 <EarthSatellite catalog #27424 epoch 2023-01-04 11:04:28 UTC>,
 <EarthSatellite catalog #27424 epoch 2023-01-04 20:26:05 UTC>,
 <EarthSatellite catalog #27424 epoch 2023-01-04 20:26:05 UTC>,
 <EarthSatellite catalog #27424 epoch 20

Plot satellite orbits historically based on TLE and using swath width find intersections of all three satellites within a certain time distinction... convert code to a class

In [9]:
analyzer = HistoricalOrbitAnalyzer(satellites, satellite_names)

In [10]:
analyzer.get_satellite_history('snpp')

[<EarthSatellite catalog #37849 epoch 2023-01-01 04:56:14 UTC>,
 <EarthSatellite catalog #37849 epoch 2023-01-01 11:40:12 UTC>,
 <EarthSatellite catalog #37849 epoch 2023-01-01 13:17:29 UTC>,
 <EarthSatellite catalog #37849 epoch 2023-01-01 19:39:39 UTC>,
 <EarthSatellite catalog #37849 epoch 2023-01-01 21:34:17 UTC>,
 <EarthSatellite catalog #37849 epoch 2023-01-02 06:26:36 UTC>,
 <EarthSatellite catalog #37849 epoch 2023-01-02 11:17:06 UTC>,
 <EarthSatellite catalog #37849 epoch 2023-01-02 14:58:03 UTC>,
 <EarthSatellite catalog #37849 epoch 2023-01-02 19:21:19 UTC>,
 <EarthSatellite catalog #37849 epoch 2023-01-02 21:16:34 UTC>,
 <EarthSatellite catalog #37849 epoch 2023-01-03 02:02:37 UTC>,
 <EarthSatellite catalog #37849 epoch 2023-01-03 10:58:08 UTC>,
 <EarthSatellite catalog #37849 epoch 2023-01-03 14:38:50 UTC>,
 <EarthSatellite catalog #37849 epoch 2023-01-03 14:38:50 UTC>,
 <EarthSatellite catalog #37849 epoch 2023-01-03 14:38:50 UTC>,
 <EarthSatellite catalog #37849 epoch 20

In [5]:
sat = analyzer.get_satellite_history('aqua')[3589]
sattime = analyzer.get_satellite_history('aqua')[3589].epoch
sat.at(sattime).is_sunlit(load('de421.bsp'))
sats = analyzer.get_satellite_history('aqua')[slice(3588,3590)]

In [31]:
sen = analyzer.get_satellite_history('sentinel2b')[3324:3326]
sen

[<EarthSatellite catalog #42063 epoch 2025-03-15 14:32:50 UTC>,
 <EarthSatellite catalog #42063 epoch 2025-03-15 19:34:56 UTC>]

In [30]:
noaa = analyzer.get_satellite_history('noaa20')[3266:3268]
noaa

[<EarthSatellite catalog #43013 epoch 2025-03-14 13:53:32 UTC>,
 <EarthSatellite catalog #43013 epoch 2025-03-14 20:39:31 UTC>]

In [6]:
sats

[<EarthSatellite catalog #27424 epoch 2025-03-14 13:40:04 UTC>,
 <EarthSatellite catalog #27424 epoch 2025-03-14 20:04:11 UTC>]

In [7]:
geocentric = sat.at(sattime)

In [8]:
sp = wgs84.subpoint_of(geocentric)

In [ ]:
# analyzer.get_satellite_history('terra')
# terra_track = analyzer.ground_track('terra')
aqua_track = analyzer.ground_track('aqua')
# landsat8_track = analyzer.ground_track('landsat8')
# sentinel2a_track = analyzer.ground_track('sentinel2a')
noaa20_track = analyzer.ground_track('noaa20')
# landsat9_track= analyzer.ground_track('landsat9')
# noaa21_track = analyzer.ground_track('noaa21')
sentinel2c_track = analyzer.ground_track('sentinel2c')

In [ ]:
print(aqua_track['lats'][120])
print(noaa20_track['lats'][110])

In [ ]:
if not -45 <= aqua_track['lats'][120] <= 45 and -45 <= noaa20_track['lats'][120] <= 45:
    print('outside of bound')

In [ ]:
gt = groundtrack_intersections(aqua_track, noaa20_track)
gt

In [ ]:
lambda x: x[0]

In [ ]:
sentinel2c_track['times']

In [ ]:
l = (-45, 45)

In [ ]:
l[0]

In [ ]:
l[1]

In [ ]:
test  =  (1, 2, 3, 4, 5, 6, 7)

In [ ]:
t, _, _, _, _, _, _ = test

In [ ]:
groundtrack_intersections(aqua_track, noaa20_track)
